## 実験的にコードを実行したい場合はこのファイルで

In [29]:
import argparse
import pathlib
import copy
import numpy as np
import sklearn.model_selection
from torch import utils
import torch
import torchvision
import torchvision.transforms.functional
from torchvision import transforms
import os

In [2]:
data_dir = './dogs-vs-cats-redux-kernels-edition'
train_path = os.path.join(data_dir, "train")

In [4]:
# mpsを利用できるか確認
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    print('mps is available')

mps is available


In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_dir", required=True)
    parser.add_argument("--device", default="mps")
    args = parser.parse_args()

    data_dir = pathlib.Path(args.data_dir)

    print(data_dir)


if __name__ == "__main__":
    main()


In [5]:
def setup_center_crop_transform():
    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )

In [ ]:
def get_labels(dataset):
    if isinstance(dataset, torch.utils.data.Subset):
        return get_labels(dataset.dataset)[dataset.indices]
    else:
        return np.array([img[1] for img in dataset.imgs])
    

def setup_train_val_datasets(data_dir, dryrun=False):
    dataset = torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform(),
    )
    labels = get_labels(dataset)
    return labels


    

In [8]:
dataset = torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform(),
    )

if isinstance(dataset, torch.utils.data.Subset):
    get_labels(dataset.dataset)[dataset.indices]
else:
    np.array([img[1] for img in dataset.imgs])

In [10]:
np.array([img[0] for img in dataset.imgs])

array(['./dogs-vs-cats-redux-kernels-edition/test/unknown/1.jpg',
       './dogs-vs-cats-redux-kernels-edition/test/unknown/10.jpg',
       './dogs-vs-cats-redux-kernels-edition/test/unknown/100.jpg', ...,
       './dogs-vs-cats-redux-kernels-edition/test/unknown/9997.jpg',
       './dogs-vs-cats-redux-kernels-edition/test/unknown/9998.jpg',
       './dogs-vs-cats-redux-kernels-edition/test/unknown/9999.jpg'],
      dtype='<U59')

In [6]:
torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform()
    ).class_to_idx

{'cat': 0, 'dog': 1}

In [ ]:
for img in dataset.imgs:
    print(np.array(img[0]))

In [28]:
b = [1, 2, 3, 4, 5]
#listオブジェクトをイテレータオブジェクトに
b_iter = iter(b)
print(type(b_iter))
next(b_iter)

<class 'list_iterator'>


1

In [48]:
a = [1, 2, 3, 4, 5]
#listオブジェクトをイテレータオブジェクトに
iter_a = iter(a)
#nextメソッドで一つづつ取り出す
print(next(iter_a))
print(next(iter_a))
print(next(iter_a))

1
2
3


In [34]:
marks = [65, 71, 68, 74, 61]

# convert list to iterator
iterator_marks = iter(marks)

# the next element is the first element
marks_1 = next(iterator_marks)
print(marks_1)

# find the next element which is the second element
marks_2 = next(iterator_marks)
print(marks_2)

print(next(iterator_marks))

# Output: 65
#         71

65
71
68


In [11]:
l = ["みかん", "りんご", "バナナ", "パイナップル"]
it = iter(l)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

みかん
りんご
バナナ
パイナップル


In [59]:
# trainデータをtrain_val_splitし、trainのインデックスとvalのインデックをそれぞれ取得
def setup_train_val_split(labels, dryrun=False, seed=0):
    x = np.arange(len(labels))
    y = np.array(labels)
    #sklearn.model_selection.StratifiedShuffleSplit
    #データセット全体のクラスの偏りを保持しながら、データセットを分割.データの重複が許容されていると分かる.必ずしも全てのデータがvalidationのデータセットに一度使われるわけではない.
    #検証用データが重複したり，ある学習用データが学習されなかったりするので，あまり使われないイメージ
    #インスタンスの作成
    splitter = sklearn.model_selection.StratifiedShuffleSplit(
        n_splits=1, 
        train_size=0.8, 
        random_state=seed    #n_splits:分割&シャッフル回数
    )
    #上で分割したtrainデータのインデックス=train_indices,valデータのインデックス=val_indices
    #x:分割するデータのインデックスの配列,y:それに対応するラベルの配列
    #next()でsplitter.split(x, y)から要素を取り出す
    train_indices, val_indices = next(splitter.split(x, y))
    #dryrun=True→ランダムに10個run
    if dryrun:
        train_indices = np.random.choice(train_indices, 10, replace=False)  #train_indicesを更新
        val_indices = np.random.choice(val_indices, 10, replace=False)

    return train_indices, val_indices

# transform
def setup_center_crop_transform():
    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )


#データ拡張
def setup_crop_flip_transform():
    """
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    指定した平均, 標準偏差でTensorを正規化
    平均、標準偏差はresnet50のデフォルトの値を使用
    https://discuss.pytorch.org/t/what-is-the-correct-pytorch-resnet50-input-normalization-intensity-range/147540
    """
    return transforms.Compose(
        [
            transforms.RandomResizedCrop(224),    # random crop
            transforms.RandomHorizontalFlip(),    # random flip:画像を左右反転
            transforms.ToTensor(),  # 画像データをtensor形式に
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])   
        ]
    )


# trainデータの正解ラベル(dog=1,cat=0)を返す
def get_labels(dataset):
    # 結局このif文は何がしたいのか分からない、このif文はfalseになる
    if isinstance(dataset, torch.utils.data.Subset):
        return get_labels(dataset.dataset)[dataset.indices]
    else:
        #traiデータの正解ラベルを全て１次元配列で出力
        return np.array([img[1] for img in dataset.imgs])  # torchvision.datasets.ImageFolder.imgs : List of (image path, class_index) tuples


#setup_train_val_splitで分割したデータをdatasetに変換
def setup_train_val_datasets(data_dir, dryrun=False):
    #torchvision.datasets.ImageFolderを使用してデータセットの作成
    dataset = torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),   #trainデータの入っているディレクトリのパス
        transform=setup_center_crop_transform()   #transformを指定することによって画像に対する前処理をすることができる
    )
    # labels=正解ラベルの配列
    labels = get_labels(dataset)
    # setup_train_val_split
    train_indices, val_indices = setup_train_val_split(labels, dryrun)

    #訓練データセットと検証データセットにデータセットを分割する際はSubsetクラスを用いる
    #元のデータセットから指定したインデックスだけ使用するデータセットを作成できる.学習及びテストに使用するインデックスを予め作成しておくことで、データセットを分割できる.
    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    val_dataset = torch.utils.data.Subset(dataset, val_indices)
    return train_dataset, val_dataset


train_dataset, val_dataset = setup_train_val_datasets(
        data_dir
    )

train_dataset = copy.deepcopy(
        train_dataset
    )  # transformを設定した際にval_datasetに影響したくない

print(train_dataset.dataset)

# train_dataset.dataset.transfrom = setup_crop_flip_transform()
train_dataset.dataset.transform = setup_crop_flip_transform()

print(train_dataset.dataset.transform)
print(train_dataset)

Dataset ImageFolder
    Number of datapoints: 25000
    Root location: ./dogs-vs-cats-redux-kernels-edition/train
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear), antialias=None)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [52]:
def setup_center_crop_transform():
    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )

dataset = torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "test")   #trainデータの入っているディレクトリのパス
        , transform=setup_center_crop_transform()
    )
image_ids = [
    os.path.splitext(os.path.basename(path))[0] for path, _ in dataset.imgs   # torchvision.datasets.ImageFolder.imgs : List of (image path, class_index) tuples
]
dataset.imgs
image_ids.index('3')
dataset.imgs[4723]

def my_index(l, x, default=False):
    if x in l:
        return l.index(x)
    else:
        return default
    
my_index(image_ids, '4')

5834

In [28]:
len([os.path.splitext(os.path.basename(path)) for path, _ in dataset.imgs])

12500

In [30]:
dataset = torch.utils.data.Subset(dataset, range(0, 100))  #
image_ids = image_ids[:100]
len(image_ids)

100

In [31]:
dataset

In [32]:
pathlib.Path().cwd()

PosixPath('/Users/matsumoto/Documents/work/kaggle/dogs_vs_cats')

In [11]:
from torch import nn
model = nn.Linear(1, 1)
model.parameters()

<generator object Module.parameters at 0x122e1cc10>

In [28]:
#yamlを読み込んで**kwargs（キーワード引数）で取り込む
import yaml
path = './config.yaml'
with open(path, 'r') as f:
    config = yaml.safe_load(f)
print(type(config), '--> configは辞書型')
#**kwargs : 辞書型のキーワード引数
#func(**辞書型オブジェクトの名前)
def func(**kwargs):
    print(kwargs)
    #これはダメ
    #print(**kwargs)

func(**config)
func(**config['optimizer'])
func(**config['optimizer']['SGD'])  #func(name='SGD', lr=0.05, momentum=0.9)と同じ

#モデルば適当にtorch.nn.Linear(1, 1)
model = torch.nn.Linear(1, 1)
def make_optimizer(params, name, **kwargs):
    return torch.optim.__dict__[name](params, **kwargs)
params = model.parameters()
make_optimizer(params, **config['optimizer']['SGD'])
# 上記はmake_optimizer(params, name='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001)と同じ

<class 'dict'> --> configは辞書型
{'optimizer': {'SGD': {'name': 'SGD', 'lr': 0.01, 'momentum': 0.9, 'weight_decay': 0.0001}, 'Adam': {'name': 'Adam', 'lr': 0.05}}, 'scheduler': {'cosineannealing': {'name': 'CosineAnnealingLR'}}, 'architecture': {'resnet50': {'name': 'resnet50', 'weights': 'IMAGENET1K_V2'}}, 'test': [{'a': None, 'name': 'a'}, {'b': None, 'name': 'b'}, {'c': None, 'name': 'c'}]}
{'SGD': {'name': 'SGD', 'lr': 0.01, 'momentum': 0.9, 'weight_decay': 0.0001}, 'Adam': {'name': 'Adam', 'lr': 0.05}}
{'name': 'SGD', 'lr': 0.01, 'momentum': 0.9, 'weight_decay': 0.0001}
Linear(in_features=1, out_features=1, bias=True)


In [119]:
import yaml
path = './config.yaml'
with open(path, 'r') as f:
    config = yaml.safe_load(f)
def func(**kwargs):
    print(kwargs)

i = config['optimizer_v2']['target']
func(**config['optimizer_v2']['ourputs'][i])
func(**config['optimizer_v3'])

#モデルば適当にtorch.nn.Linear(1, 1)
model = torch.nn.Linear(1, 1)
def make_optimizer(params, name, **kwargs):
    return torch.optim.__dict__[name](params, **kwargs)
params = model.parameters()
# make_optimizer(params, **config['optimizer_v2'])

if 'SGD' in config['optimizer_v3']:
    print('SGD')

{'name': 'SGD', 'lr': 0.01, 'momentum': 0.9}
{'SGD': {'name': 'SGD', 'lr': 0.01, 'momentum': 0.9}, 'Adam': {'name': 'Adam'}}
SGD


In [61]:
t = (1, 2, 3, 4, 1, 2)
type(t)
t[0]
t.count(1)

2

In [139]:
def sub(x,y):
    print(x-y)

def sum(a,b,x,y):
    sub(x,y)
    return a + b

sum(a=1,b=2,x=3,y=2)

1


3

In [80]:
#ただepoch回実行したい場合
print(list(range(3)))
for epoch in range(3):
    print('hello world')

[0, 1, 2]
hello world
hello world
hello world


In [24]:
import yaml
path = './config.yaml'
with open(path, 'r') as f:
    config = yaml.safe_load(f)
def func(**kwargs):
    print(kwargs)


# torch.optim.__dict__[name](params, **kwargs)

func(**config['architecture']['resnet50'])

target_scheduler='CosineAnnealingLR'
torch.optim.lr_scheduler.__dict__[target_scheduler]



{'name': 'resnet50', 'weights': 'torchvision.models.ResNet50_Weights.IMAGENET1K_V2'}


torch.optim.lr_scheduler.CosineAnnealingLR